In [ ]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
import base64

# Load the .env file
load_dotenv(dotenv_path='api.env')

# Fetch environment variables
channel1_api_key = os.getenv('CHANNEL1_API_KEY')
channel2_api_key = os.getenv('CHANNEL2_API_KEY')

# Define API details
channels = {
    "Channel1": {
        "url": f"https://api.thingspeak.com/channels/2561136/feeds.json?api_key={channel1_api_key}",
        "fields": ["created_at", "field2"],
        "rename": {
            "created_at": "timestamp",
            "field2": "Body_temp"
        }
    },
    "Channel2": {
        "url": f"https://api.thingspeak.com/channels/2561145/feeds.json?api_key={channel2_api_key}",
        "fields": ["created_at", "field2", "field3"],
        "rename": {
            "created_at": "timestamp",
            "field2": "heartRate",
            "field3": "SP02"
        }
    }
}

# Function to fetch and transform data
def fetch_and_transform_data(channel_url, fields, rename_columns):
    try:
        response = requests.get(channel_url)
        response.raise_for_status()
        data = response.json()

        if 'feeds' not in data:
            print("Error: 'feeds' key not found in the API response.")
            return pd.DataFrame()

        df = pd.DataFrame(data['feeds'])
        df = df[fields]
        df.rename(columns=rename_columns, inplace=True)
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        for column in df.columns:
            if column != 'timestamp':
                df[column] = pd.to_numeric(df[column], errors='coerce').round(2)

        return df

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {channel_url}: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"An error occurred: {e}")
        return pd.DataFrame()

# Function to generate a new AES key
def generate_aes_key():
    return os.urandom(32)  

# Function to encrypt a message using AES
def encrypt_message(key, plaintext):
    iv = os.urandom(16)  # 128-bit IV
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(plaintext.encode()) + encryptor.finalize()
    return base64.b64encode(iv + ciphertext).decode()

# Fetch and transform data for both channels
df_channel_1 = fetch_and_transform_data(channels['Channel1']['url'], channels['Channel1']['fields'], channels['Channel1']['rename'])
df_channel_2 = fetch_and_transform_data(channels['Channel2']['url'], channels['Channel2']['fields'], channels['Channel2']['rename'])

# Display data before encryption
print("Channel 1 Data Before Encryption:")
print(df_channel_1.head())

print("\nChannel 2 Data Before Encryption:")
print(df_channel_2.head())

# Encrypt the data
key = generate_aes_key()
df_channel_1['Body_temp'] = df_channel_1['Body_temp'].apply(lambda x: encrypt_message(key, str(x)))
df_channel_2['heartRate'] = df_channel_2['heartRate'].apply(lambda x: encrypt_message(key, str(x)))
df_channel_2['SP02'] = df_channel_2['SP02'].apply(lambda x: encrypt_message(key, str(x)))

# Display data after encryption 
print("\nChannel 1 Data After Encryption:")
print(df_channel_1[['timestamp', 'Body_temp']].head())

print("\nChannel 2 Data After Encryption:")
print(df_channel_2[['timestamp', 'heartRate', 'SP02']].head())


In [ ]:
# Function to decrypt an encrypted message using AES
def decrypt_message(key, ciphertext):
    ciphertext = base64.b64decode(ciphertext)
    iv = ciphertext[:16]  # Extract the IV from the first 16 bytes
    actual_ciphertext = ciphertext[16:]  # The rest is the encrypted message
    cipher = Cipher(algorithms.AES(key), modes.CFB(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    plaintext = decryptor.update(actual_ciphertext) + decryptor.finalize()
    return plaintext.decode()

# Decrypt the data
df_channel_1['Decrypted_Body_temp'] = df_channel_1['Body_temp'].apply(lambda x: decrypt_message(key, x))
df_channel_2['Decrypted_heartRate'] = df_channel_2['heartRate'].apply(lambda x: decrypt_message(key, x))
df_channel_2['Decrypted_SP02'] = df_channel_2['SP02'].apply(lambda x: decrypt_message(key, x))

# Display the decrypted data
print("\nChannel 1 Data After Decryption:")
print(df_channel_1[['timestamp', 'Decrypted_Body_temp']].head())

print("\nChannel 2 Data After Decryption:")
print(df_channel_2[['timestamp', 'Decrypted_heartRate', 'Decrypted_SP02']].head())
